In [2]:
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
# Parâmetros
image_size = (64, 64)  
batch_size = 32
epochs = 20

# Caminhos para os dados
train_data_dir = 'imagens/train/'
test_data_dir = 'imagens/test/'

In [4]:
# Configuração para utilizar a GPU se disponível
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Usando GPU:", physical_devices[0])
else:
    print("GPU não encontrada, utilizando CPU")

GPU não encontrada, utilizando CPU


In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

KeyboardInterrupt: 

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))  

# Compilação do modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Resumo do modelo
model.summary()

In [ ]:
# Treinamento do modelo
history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    verbose=1
)

In [ ]:
# Salvar o modelo após o treinamento
model.save('modelo_libras_CNN.h5')

In [ ]:
import matplotlib.pyplot as plt

# Plotando a acurácia
plt.plot(history.history['accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.show()

# Plotando a perda
plt.plot(history.history['loss'])
plt.title('Perda do modelo')
plt.ylabel('Perda')
plt.xlabel('Época')
plt.show()


In [ ]:
# Avaliação do modelo
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print('Acurácia no conjunto de teste:', test_acc)

# Mapeamento das classes
class_indices = train_generator.class_indices
print("Class Indices:", class_indices)

# Exemplo de uma previsão
img_path = 'imagens/test/A/1.png'  # Exemplo de caminho da imagem
img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Fazer a previsão
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])

# Obter o nome da classe prevista
classes = list(class_indices.keys())
predicted_label = classes[predicted_class]
print("Predicted Label:", predicted_label)


In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import time

# Carregar o modelo treinado
model = load_model('modelo_libras.h5')

# Função para exibir o resultado da classificação na câmera
def display_classification(frame, predicted_letter):
    # Adicionar o texto da classificação na imagem
    cv2.putText(frame, f'Classificacao: {predicted_letter}', (20, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    
    # Exibir a imagem com a classificação
    cv2.imshow('Classificacao de Libras', frame)

# Função para pré-processamento da imagem
def preprocess_image(img):
    img = cv2.resize(img, (64, 64))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Função para exibir o resultado da classificação
def display_prediction(img, label):
    #cv2.putText(img, label, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.imshow('Camera', img)

# Inicializar a câmera
cap = cv2.VideoCapture(0)

# Último tempo de captura
last_capture_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar o frame da camera")
        break
    
    # Mostrar a câmera com um label inicial
    display_prediction(frame, '"Q" -> sair')

    # Tempo atual
    current_time = time.time()
    
    # Verificar se 0.5 segundos passaram desde a última captura
    if current_time - last_capture_time >= 0.5:
        last_capture_time = current_time
        
        # Tirar uma foto (frame) e pré-processar
        photo = preprocess_image(frame)

        # Fazer a previsão usando o modelo
        prediction = model.predict(photo)
        predicted_class = np.argmax(prediction)
        
        # Mapear a classe prevista para a letra correspondente
        classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W']
        predicted_letter = classes[predicted_class]

        # Exibir o resultado da classificação na câmera usando o OpenCV
        display_classification(frame, predicted_letter)
        
        # Extrair a imagem pré-processada para visualização
        preprocessed_img = (photo[0] * 255).astype('uint8')  # Remover a dimensão adicional e reescalar
        cv2.imshow('Imagem Pre-processada', cv2.resize(preprocessed_img, (256, 256)))  # Redimensionar para facilitar a visualização

    # Fechar a câmera ao pressionar a tecla "q"
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━